# Import Resources

In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import numpy as np
from sshtunnel import SSHTunnelForwarder
import pandas as pd
import pymysql
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV


# **DATA BASE**

In [2]:
## VARIABLE DEFINITIONS

ssh_ip_address = ''
ssh_p_username = ''
ssh_p_password = ""
ssh_port = 22
ssh_remote_bind_address = 3306
db_server_ip = ''
db_server_port = 3306
db_user = ''
db_user_password = ''
db_name = ''

## SSH CONNECTION

tunnel = SSHTunnelForwarder((ssh_ip_address, 22), ssh_password=ssh_p_password, ssh_username=ssh_p_username,
                            remote_bind_address=('127.0.0.1', 3306))
tunnel.start()
# Establish an SSH tunnel to the remote server using the specified IP address, port, username, and password
# The tunnel will bind to the local address '127.0.0.1' and port 3306

# MYSQL CONNECTION
db = pymysql.connect(host='localhost', port=tunnel.local_bind_port, user=ssh_p_username,
                          passwd=db_user_password, db=db_name)
# Establish a connection to the MySQL server using the local bind port of the SSH tunnel, username, password, and database name


# Create a cursor object to execute SQL queries on the database
cur = db.cursor()

In [3]:
# SQL query to select all data from the table
query = "SELECT * FROM TI_PV_main;"
# Execute the query
cur.execute(query)
# Fetch all the rows
rows1 = cur.fetchall()

# SQL query to select all data from the table
query = "SELECT * FROM TI_PV_branch;"
# Execute the query
cur.execute(query)
# Fetch all the rows
rows2 = cur.fetchall()


df1 = pd.DataFrame(rows1)
df2 = pd.DataFrame(rows2)

# Drop two columns from the first DataFrame
columns_to_drop = [0, 1]
df2 = df2.drop(columns=columns_to_drop)

# # Concatenate the DataFrames
df_c = pd.concat([df1, df2], axis=1)
Labels=df_c[df_c.columns[1]].tolist()

X= df_c.drop(df_c.columns[:2], axis=1)

# Modify labels
for i in range(0,len(Labels)):
    if Labels[i] =='No-Anomaly':
        pass
    else:
        Labels[i]='Anomaly'


# Create an instance of the MinMaxScaler
scaler = MinMaxScaler()
# Normalize the data in the DataFrame
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
y=Labels



In [4]:
y=Labels

# Prepared data for TEST

In [37]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=55)

In [40]:
X_trainS=X_train.iloc[:,0:300]
X_testS=X_test.iloc[:,0:300]

In [15]:
X_test.shape

(2000, 1000)

In [6]:
# To select the top 20 most important features using mutual_info_classif:
from sklearn.feature_selection import SelectKBest, mutual_info_classif
selector = SelectKBest(score_func=mutual_info_classif, k=300) 
X_selected = selector.fit_transform(X, y)
#X_selected=X

In [7]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_selected, y, test_size=0.1, random_state=55)

# Rebuild model from CSV file

In [19]:
import pandas as pd

# Load the Grid Search results from the CSV file
df = pd.read_csv("GS_BaggingClassifier1.csv")
# Find the index of the best model based on a metric (e.g., 'mean_test_score')
best_model_index = df['mean_test_score'].idxmax()
# Get the parameters of the best model
best_params_Bagging = eval(df.loc[best_model_index, 'params'])


# Load the Grid Search results from the CSV file
df = pd.read_csv("GS_results_RandomforestClassifier3.csv")
# Find the index of the best model based on a metric (e.g., 'mean_test_score')
best_model_index = df['mean_test_score'].idxmax()
# Get the parameters of the best model
best_params_forest = eval(df.loc[best_model_index, 'params'])

In [20]:
#Stiven results
#0.696
#0.890 GS_BaggingClassifier1.csv
#0.67455
#0.7487
#0.87  
#0.86  
#0.886 GS_results_RandomforestClassifier3.csv
#0.86  

df['mean_test_score'].max()


0.8866666666666667

In [21]:
best_params_Bagging

{'base_estimator__max_depth': 3,
 'base_estimator__min_samples_leaf': 2,
 'base_estimator__min_samples_split': 4,
 'max_features': 0.5,
 'max_samples': 0.8,
 'n_estimators': 50}

In [22]:
best_params_forest

{'max_depth': 15,
 'min_samples_leaf': 3,
 'min_samples_split': 4,
 'n_estimators': 200}

In [44]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# Create a base estimator (e.g., DecisionTreeClassifier) using the specified max_depth and min_samples_split
base_estimator = DecisionTreeClassifier(max_depth=best_params_Bagging['base_estimator__max_depth'],
                                        min_samples_split=best_params_Bagging['base_estimator__min_samples_split'],
                                        min_samples_leaf=best_params_Bagging['base_estimator__min_samples_leaf'])

# Create the BaggingClassifier using the base_estimator and other hyperparameters
model_Gradient = BaggingClassifier(base_estimator=base_estimator,
                                   max_features=best_params_Bagging['max_features'],
                                   max_samples=best_params_Bagging['max_samples'],
                                   n_estimators=best_params_Bagging['n_estimators'])

# Train the model with your training data
model_Gradient.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=3,
                                                        min_samples_leaf=2,
                                                        min_samples_split=4),
                  max_features=0.5, max_samples=0.8, n_estimators=50)

In [46]:
# Calculate the accuracy of the classifier
y_pred=model_Gradient.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6955


In [47]:
from sklearn.ensemble import RandomForestClassifier

# Assuming you have the best_params_forest dictionary for the RandomForestClassifier
best_params_forest = {'max_depth': 15, 'min_samples_leaf': 3, 'min_samples_split': 4, 'n_estimators': 200}
# Extract the 'n_estimators' value from best_params_forest
n_estimators_value = best_params_forest['n_estimators']
# Remove 'n_estimators' from best_params_forest
del best_params_forest['n_estimators']
# Create the RandomForestClassifier using the best_params_forest and n_estimators_value
model_Forest = RandomForestClassifier(n_estimators=n_estimators_value, **best_params_forest)
# Train the model with your training data
model_Forest.fit(X_train, y_train)

RandomForestClassifier(max_depth=15, min_samples_leaf=3, min_samples_split=4,
                       n_estimators=200)

In [48]:
# Calculate the accuracy of the classifier
y_pred=model_Forest.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.77525


# Read model from .pkl File

In [49]:
import pickle
model_file = 'best_random_forest__1_Daniel.pkl'
# Load the model from the file
with open(model_file, 'rb') as f:
    loaded_model = pickle.load(f)



In [50]:
from sklearn.metrics import accuracy_score
# Calculate the accuracy of the classifier
y_pred=loaded_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9365
